# Setting up access to the Pele API

#### *Pele* is a RESTful API which provides access to the contents of the data product catalog.

#### This notebook will walk you through the process of registering for pele access, storing your credentials locally, and instantiating a handle which will automatically use these credentials to access Pele.

### Set the correct value of your Mozart host

#### Set the correct Mozart hostname or ip address in the `mozart_host` variable below

In [7]:
import requests, json, getpass
from requests.auth import HTTPBasicAuth
import urllib3

urllib3.disable_warnings()

# set the base url to interact with the goddess, Pele
mozart_host = '137.78.250.114'
base_url = f'https://{mozart_host}/pele/api/v0.1'
print("Using base url {}.".format(base_url))

Using base url https://137.78.250.114/pele/api/v0.1.


### Set your username (email) and password

In [8]:
user = input("Enter email address then press <Enter>: ")
print("Enter password then press <Enter>.")
print("*ATTENTION!!! DON'T USE YOUR JPL PASSWORD HERE. MAKE SOMETHING UP FOR THIS TEST.*")
password = getpass.getpass()

print(f'Using username "{user}" and password: "{password}"'.format(password))

Enter email address then press <Enter>:  alexander.e.dunn@jpl.nasa.gov


Enter password then press <Enter>.
*ATTENTION!!! DON'T USE YOUR JPL PASSWORD HERE. MAKE SOMETHING UP FOR THIS TEST.*


 ··········


Using username "alexander.e.dunn@jpl.nasa.gov" and password: "Majestic12"


### Register with your email/password

In [9]:
r = requests.post(base_url + '/register', data={'email': user, 'password': password}, verify=False)

# expect 201 (created)
print("status code: {}".format(r.status_code))
print("content: {}".format(r.content.decode()))
assert r.status_code == 201

status code: 500
content: {"success": false, "message": "Registration failed. Please contact support.", "email": null, "id": null}



AssertionError: 

## Verify your account

#### You will receive an email, containing a verification code. For example:

```
Use your verification code below to verify your Pele API account at http://localhost:8877/api/v0.1/:

ffa8d18b-f581-44bf-8864-b52a2cd8e7b6
```

#### The following cell will prompt you for that verification code:

In [10]:
# prompt for verification code
ver_code = input("Enter the verification code:")

# verify
r = requests.post(base_url + '/verify', data={'email': user, 'verification_code': ver_code}, verify=False)

# expect 200
print("status code: {}".format(r.status_code))
print(json.dumps(r.json(), indent=2))
assert r.status_code == 200

Enter the verification code: arst


status code: 401
{
  "success": null,
  "message": "Invalid verification code"
}


AssertionError: 

## Logging in to retrieve an API token

#### Once your user registration is verified, you can then log into the Pele API which will provide you with an API token (valid for a period of time). The API token will allow you to make requests of the Pele API:

In [18]:
r = requests.post(base_url + 'pele/dataset/S1-TIMESERIES-MINTPY/dataset_ids', auth=HTTPBasicAuth(user, password), verify=False)

# expect 200
print("status code: {}".format(r.status_code))
print(json.dumps(r.json(), indent=2))

# extract API token
token = r.json()['token']
print("token: {}".format(token))
assert r.status_code == 200

status code: 404


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Making a restricted API call using your token

#### When making a Pele API call that is restricted to authenticated users, pass your API token in a header called `X-API-KEY`:

In [12]:
r = requests.get(base_url + '/test/echo', params={'echo_str': 'hello world'}, headers={'X-API-KEY': token}, verify=False)

# expect 200
print("status code: {}".format(r.status_code))
print(json.dumps(r.json(), indent=2))
assert r.status_code == 200

status code: 200
{
  "success": true,
  "message": "hello world"
}


## Detecting expiration of your API token

#### Your API token will expire after some time (the default is 24 hours). The example below shows that when your token has expired, you will receive a `401` status code error with the error message: `Expired token. Reauthentication required.`:

```
In [1]: r = requests.get(base_url + '/test/echo', params={'echo_str': 'hello world'}, headers={'X-API-KEY': token})

In [2]: r.status_code
Out[2]: 401

In [3]: r.json()
Out[3]: 
{u'authenticated': False,
 u'message': u'Expired token. Reauthentication required.'}
```

#### At this point, you will have to login again to refresh your API token (see [login to get API token](#login-to-get-API-token) above).

## Using the Pele requests client automatically maintain authentication

If you will be running scripts that will interact with the Pele API, you will need to utilize a method for automatically logging into the Pele REST API to request the API token and to refresh the token should the token expire during the script's execution.

The Pele requests client can utilize the `.netrc` file to automate this for you.

Here we populate your .netrc:

In [13]:
from urllib.parse import urlparse
import getpass, os, stat

system_username = getpass.getuser()
# parse url to get netloc component
pr = urlparse(base_url)

print("netloc: {}".format(pr.netloc))

# get home directory
stream = os.popen('ls -d ~')
home_dir = stream.read().strip()

# create .netrc
print(f"Writing {home_dir}/.netrc")
with open(f"{home_dir}/.netrc", 'a') as f:
    f.write(f"machine {pr.netloc} login {user} password {password}\nmacdef init\n\n")

# fix perms
os.chmod(f"{home_dir}/.netrc", stat.S_IRUSR | stat.S_IWUSR)

print('Complete')

netloc: 137.78.250.114
Writing /home/jovyan/.netrc
Complete


## Testing the Pele client

If you receive a successful response, you will now be able to use the `pele` client to access the Pele API without manual authentication

In [14]:
from pele.lib.client import PeleRequests

# instantiate PeleRequests object
print(f"Base URL {base_url}")
pr = PeleRequests(base_url, verify=False)

# now use like requests module (`request()`, `get()`, `head()`, `post()`, `put()`, `delete()`, `patch()`)
r = pr.get(base_url + '/test/echo', params={'echo_str': 'hello world'})

# expect 200
print("status code: {}".format(r.status_code))
print(json.dumps(r.json(), indent=2))
assert r.status_code == 200

Base URL https://137.78.250.114/pele/api/v0.1
status code: 200
{
  "success": true,
  "message": "hello world"
}


## Dealing with rate-limited API calls

#### The Pele REST API rate-limits calls to prevent DoS-like access to the backend database:

In [ ]:
for i in range(20):
    r = pr.get(base_url + '/test/echo', params={'echo_str': f'({i}) hello world'})
    print("({}) status code: {}".format(i, r.status_code))
    print(json.dumps(r.json(), indent=2))
    r.raise_for_status()

#### To mitigate this, use the `backoff` python module to apply exponential backoff when making numerous requests.

In [ ]:
!pip install backoff

In [ ]:
import backoff
from requests.exceptions import HTTPError

@backoff.on_exception(backoff.expo, HTTPError, max_tries=3, max_value=5)
def echo(i):
    r = pr.get(base_url + '/test/echo', params={'echo_str': f'({i}) hello world'})
    print("({}) status code: {}".format(i, r.status_code))
    print(json.dumps(r.json(), indent=2))
    r.raise_for_status()
    
for i in range(20):
    echo(i)

<font size="1">This notebook is compatible with NISAR Jupyter Server Stack v1.4 and above</font>